# pytorch version

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import torch.utils.data.dataloader as dataloader
import os

In [3]:
train_df= pd.read_csv("Training_set.csv")
test_df= pd.read_csv("Testing_set.csv")
train_df.nunique()

filename    6499
label         75
dtype: int64

In [4]:
train_img= r"/Users/nguyenthephong/Desktop/Okay this is ML serious/project for practicing/butterfly ah ah folder/train"
test_img=r"/Users/nguyenthephong/Desktop/Okay this is ML serious/project for practicing/butterfly ah ah folder/test"

#method for get the class for the train path
def set_class(path,df):
    for images_name in os.listdir(path):
        for index,row in df.iterrows():
            if row["filename"]==images_name:
                class_img= row["label"]
                if not os.path.exists(os.path.join(path,class_img)):
                    os.makedirs(os.path.join(path,class_img))
                shutil.move(os.path.join(path,images_name),os.path.join(path,class_img, images_name))

# set_class(train_img,train_df)

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
train_transform =T.Compose([T.Resize(512),
                      T.ToTensor(),
                      T.RandomHorizontalFlip(p=0.5),
                      T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])

test_transform =T.Compose([T.Resize(512),
                      T.ToTensor(),
                      T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])

In [7]:
train_path= ImageFolder(root=train_img,transform=train_transform)
train_path[0][0].shape

torch.Size([3, 512, 512])

In [8]:
train_loader = dataloader.DataLoader(train_path, shuffle=True, batch_size=32)

In [9]:
class BruhNN(nn.Module):
    def __init__(self,color_channel,classes):
        super(BruhNN, self).__init__()
        self.Convlayer= nn.Sequential(
            nn.Conv2d(color_channel,256,kernel_size=3),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(256,128,kernel_size=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(128,64,kernel_size=3),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64,128,kernel_size=1),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,256,kernel_size=1)
        )
        self.flattenlayer= nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*256*256,512),
            nn.LeakyReLU(),
            nn.Linear(512,1024),
            nn.LeakyReLU(),
            nn.Linear(1024,classes)
        )
    def forward(self,x):
        return self.flattenlayer(self.Convlayer(x))

In [10]:
model= BruhNN(3,75)
param=0
for paras in model.parameters():
    param += paras.flatten().shape[0]

param

101422347

In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr=0.01)
loss=nn.CrossEntropyLoss()
wrong_stored=[]
model.to(device)
for epoch in range(10):
    model.train()
    for i,(inputs,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        predicted=model(inputs.to(device))
        wrong=loss(predicted,labels.to(device))
        wrong.backward()
        optimizer.step()
        wrong_stored.append(wrong)

wrong_stored